In [1]:
import string
import re
import os
import nltk
import pandas as pd
import numpy as np
import json
import tensorflow as tf
from tensorflow import keras
SEED = 1013
np.random.seed(SEED)
#nltk.download('stopwords')
from nltk.tokenize import TweetTokenizer
from nltk.corpus import stopwords, twitter_samples 
from stance_utils import *
#from parameters import *
from nltk.stem import PorterStemmer
from sklearn.metrics import classification_report
from sklearn.feature_extraction.text import CountVectorizer
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras import Sequential
#from tensorflow.keras.layers import Dropout,Concatenate,Dense, Embedding, SpatialDropout1D, Flatten, GRU, Bidirectional, Conv1D,MaxPooling1D

from tensorflow.keras.layers import RNN, Dropout,Concatenate,Dense, Embedding,LSTMCell, LSTM, SpatialDropout1D, Flatten, GRU, Bidirectional, Conv1D, Input,MaxPooling1D
from sklearn.model_selection import train_test_split
from tensorflow.keras import Model
from sklearn.model_selection import StratifiedKFold
stemmer = PorterStemmer()
tokenizer = TweetTokenizer(preserve_case=False, strip_handles=True, reduce_len=True)
stopwords_english = stopwords.words('english')
from sklearn.preprocessing import LabelEncoder
import keras.backend as K
from keras.layers import Lambda

In [2]:
classes = {'FAVOR': np.array([1, 0, 0]), 'AGAINST': np.array([0, 1, 0]), 'NONE': np.array([0, 0, 1])}
classes_ = np.array(['FAVOR', 'AGAINST', 'NONE'])

In [3]:
train_data_file_m = '/data/parush/stance_mohammed/train.txt'
test_data_file_m = '/data/parush/stance_mohammed/test.txt'
TARGETS_m = [ 'Atheism','Climate Change is a Real Concern', 'Feminist Movement','Hillary Clinton', 'Legalization of Abortion' ]

In [4]:
def train_and_test(train_file, test_file, target):
    
    sentence_maxlen = 0
    target_maxlen = 0
    x_s_token = []
    x_t_token = []
    y_train = []

    
    with open(train_file, 'r') as trainfile:
        for line in trainfile: 
            line = line.replace('#SemST', '').strip()
            line = line.split('\t')
            
            
            if line[0].strip() != 'ID' and target in line[1].strip():
                tweet = line[2]
                tweet = process_tweet(tweet)
                if len(tweet) > sentence_maxlen:
                    sentence_maxlen = len(tweet)
                x_s_token.append(tweet)
                target_ = line[1].strip().lower().split()
                if len(target_) > target_maxlen:
                    target_maxlen = len(target_)
                x_t_token.append(target_)
                y_train.append(classes[line[3].strip()])
    

                               
    
    x_s_test_token = []
    x_t_test_token = []
    y_test = []
    with open(test_file, 'r') as testfile:
        for line in testfile:
            line = line.replace('#SemST', '').strip()
            line = line.split('\t')
        

            
            if line[0] != 'ID' and target in line[1].strip():
                tweet = line[2]
                tweet = process_tweet(tweet)
                if len(tweet) > sentence_maxlen:
                    sentence_maxlen = len(tweet)
                x_s_test_token.append(tweet)
                target_ = line[1].strip().lower().split()
                if len(target_) > target_maxlen:
                    target_maxlen = len(target_)
                x_t_test_token.append(target_)
                y_test.append(classes[line[3].strip()])


    
    return x_s_token, x_t_token, x_s_test_token, x_t_test_token, y_train, y_test, sentence_maxlen, target_maxlen

In [5]:
x_s_token, x_t_token, x_s_test_token, x_t_test_token, y_train, y_test, sentence_maxlen, target_maxlen  = train_and_test(train_data_file_m, test_data_file_m, TARGETS_m[2])

In [6]:
vocabulary = build_vocab(x_s_token + x_t_token + x_s_test_token + x_t_test_token)
vocab_size = len(vocabulary)
print("Total words in vocab are",vocab_size)

Total words in vocab are 2943


In [7]:
x_s = [tweet_to_tensor(each_s,vocabulary) for each_s in x_s_token]
x_s = pad_sequences(x_s, maxlen = sentence_maxlen, padding = 'post')
x_s_test = [tweet_to_tensor(each_s,vocabulary) for each_s in x_s_test_token]
x_s_test = pad_sequences(x_s_test, maxlen = sentence_maxlen, padding = 'post')

x_t = [tweet_to_tensor(each_s,vocabulary) for each_s in x_t_token]
x_t = pad_sequences(x_t, maxlen = sentence_maxlen, padding = 'post')
x_t_test = [tweet_to_tensor(each_s,vocabulary) for each_s in x_t_test_token]
x_t_test = pad_sequences(x_t_test, maxlen = sentence_maxlen, padding = 'post')

In [8]:
shuffle_indices = np.random.permutation(np.arange(len(y_train)))
x_s = x_s[shuffle_indices]
x_t = x_t[shuffle_indices]

y_train = np.asarray(y_train)
y_train = y_train[shuffle_indices]
y_test = np.asarray(y_test)

In [9]:
x_s = x_s[:640]
x_t = x_t[:640]
y_train= y_train[:640]





In [10]:
x_t =np.array( [x.reshape(1,18) for x in x_t[:640]])

In [11]:
x_t = tf.convert_to_tensor(
    x_t, dtype=tf.float32, dtype_hint=None, name=None
)

2021-10-15 16:26:34.961295: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /apps/cuda/cuda-11.2/lib64:/slurm/include
2021-10-15 16:26:34.961336: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2021-10-15 16:26:34.961363: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (gaivi.cse.usf.edu): /proc/driver/nvidia/version does not exist
2021-10-15 16:26:34.961985: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [12]:
h_0 = tf.convert_to_tensor(np.zeros([16, 128]).astype(np.float32))
c_0 = tf.convert_to_tensor(np.zeros([16, 128]).astype(np.float32))


In [13]:
# i_states = [h_0, c_0]
# encoder = RNN(tf.compat.v1.nn.rnn_cell.LSTMCell(128), return_state= True)
# output, c_0, h_0 = encoder(x_t[:16],initial_state = i_states)

In [22]:

inputs = x_t[:16]
lstm = LSTM(128, return_sequences=True, return_state=True)
whole_seq_output, h_0, c_0 = lstm(inputs,initial_state = [h_0, c_0])



In [15]:
h_0 = tf.convert_to_tensor(np.zeros([16, 128]).astype(np.float32))

In [16]:
#embedding_matrix = tf.random.uniform([vocab_size, 300], -0.1, 0.1) #input_size is embeddings size
embedding_matrix = get_embeddings('wikipedia',300,vocabulary)                                  

In [17]:
embedding_matrix.shape

(2943, 300)

In [19]:

inputs2 = Input(shape=(sentence_maxlen), name = 'Input')

embedded_inputs = Embedding(embedding_matrix.shape[0], embedding_matrix.shape[1], weights=[embedding_matrix], name = 'Embedding')(inputs2)
embedded_inputs2 = Dropout(0.2)(embedded_inputs)
lstm = LSTM(128, activation = 'tanh',dropout=0.3,name = 'lstm')(embedded_inputs2, initial_state = [h_0, c_0])
flat = Flatten(name = 'Flatten')(lstm)
output = (Dense(3,activation='softmax',name = 'Dense'))(flat)
model2 = Model(inputs=inputs2, outputs=output)
model2.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])    
model2.summary()


Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Input (InputLayer)           [(None, 18)]              0         
_________________________________________________________________
Embedding (Embedding)        (None, 18, 300)           882900    
_________________________________________________________________
dropout_1 (Dropout)          (None, 18, 300)           0         
_________________________________________________________________
lstm (LSTM)                  (16, 128)                 219648    
_________________________________________________________________
Flatten (Flatten)            (16, 128)                 0         
_________________________________________________________________
Dense (Dense)                (16, 3)                   387       
Total params: 1,102,935
Trainable params: 1,102,935
Non-trainable params: 0
_________________________________________________

In [25]:
model2.fit(x_s, y_train, epochs = 50, batch_size = 16, verbose=1)

Epoch 1/50
40/40 [==============================] - 2s 17ms/step - loss: 1.0211 - accuracy: 0.4953
Epoch 2/50
40/40 [==============================] - 1s 17ms/step - loss: 0.9002 - accuracy: 0.5859
Epoch 3/50
40/40 [==============================] - 1s 17ms/step - loss: 0.7107 - accuracy: 0.7000
Epoch 4/50
40/40 [==============================] - 1s 17ms/step - loss: 0.5136 - accuracy: 0.7984
Epoch 5/50
40/40 [==============================] - 1s 17ms/step - loss: 0.3200 - accuracy: 0.8781
Epoch 6/50
40/40 [==============================] - 1s 17ms/step - loss: 0.1903 - accuracy: 0.9391
Epoch 7/50
40/40 [==============================] - 1s 17ms/step - loss: 0.2363 - accuracy: 0.9187
Epoch 8/50
40/40 [==============================] - 1s 17ms/step - loss: 0.1080 - accuracy: 0.9641
Epoch 9/50
40/40 [==============================] - 1s 17ms/step - loss: 0.0890 - accuracy: 0.9688
Epoch 10/50
40/40 [==============================] - 1s 17ms/step - loss: 0.0709 - accuracy: 0.9828
Epoch 11/

In [26]:
y_pred = np.round(model2.predict(x_s_test[:272], batch_size = 16))

In [27]:
y_test.shape

(285, 3)

In [28]:
print(classification_report(y_test[:272], y_pred, digits=4, labels = [0,1]))

              precision    recall  f1-score   support

           0     0.3077    0.4000    0.3478        50
           1     0.7931    0.6389    0.7077       180

   micro avg     0.6429    0.5870    0.6136       230
   macro avg     0.5504    0.5194    0.5278       230
weighted avg     0.6876    0.5870    0.6295       230
 samples avg     0.4963    0.4963    0.4963       230



/home/p/parush/.conda/envs/nlp/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/p/parush/.conda/envs/nlp/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [21]:
# kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=SEED)
# cvscores = []

# for train, val in kfold.split(x_s, classes_[y_train.argmax(1)]): 
#     model2.fit(x_s[train], y_train[train], epochs = 5, batch_size = 16, verbose=1)
# #     scores = model2.evaluate(x_s[val], y_train[val], verbose=0)
# #     print("%s: %.2f%%" % (model2.metrics_names[1], scores[1]*100))
# #     cvscores.append(scores[1] * 100)
# # print("%.2f%% (+/- %.2f%%)" % (np.mean(cvscores), np.std(cvscores)))





In [ ]:
#embeddings_weights = get_embeddings('wikipedia',300,vocabulary)


In [ ]:
# Do we have to do this?
# x_s_ = tf.convert_to_tensor(
#     x_s_, dtype=tf.float32, dtype_hint=None, name=None
# )

In [ ]:
# i = 0
# j = 16
# while j<=640:
#     inputs = x_t[i:j]
#     lstm = LSTM(128, return_sequences=True, return_state=True)
#     whole_seq_output, h_0, c_0 = lstm(inputs,initial_state = [h_0, c_0])
#     i += 16
#     j += 16



In [ ]:
# inputs = Input(batch_shape=(16,1,18))
# lstm = LSTM(128, return_sequences=True, return_state=True)
# whole_seq_output, h_0, c_0 = lstm(inputs)
# model = Model(inputs=inputs, outputs=whole_seq_output)
# k = model.predict(x_t[:640])

In [ ]:
# final_memory_state = tf.reshape(
#     final_memory_state, (16,1,128), name=None
# )

# final_carry_state = tf.reshape(
#     final_carry_state, (16,1,128), name=None
# )


In [ ]:

# inputs2 = Input(shape=(1,sentence_maxlen,), name = 'Input')
# #embedded_inputs = Embedding(embeddings_weights.shape[0], embeddings_weights.shape[1], weights=[embeddings_weights], name = 'Embedding')(inputs2)
# #embedded_inputs2 = Dropout(0.2)(embedded_inputs)
# lstm, s1,s2 = LSTM(128,return_sequences=True,return_state=True,dropout=0.3,name = 'lstm')(inputs2, initial_state = i_states)
# #lstm = LSTM(128,return_sequences=True,dropout=0.3,name = 'lstm')(inputs2)
# flat = Flatten(name = 'Flatten')(lstm)
# output = (Dense(3,activation='softmax',name = 'Dense'))(flat)
# model2 = Model(inputs=inputs2, outputs=output)
# model2.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])    
# model2.summary()
